In [1]:
import pandas as pd
import spacy
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora
import re
import string

nlp = spacy.load('en_core_web_lg')

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/sub-stats/ds/master/4-30-to-7-29-comments.csv')
df.loc[df['body'].isnull(), 'body'] = 'Null'
df['retrieved_on'] = pd.to_datetime(df['retrieved_on'], unit='s')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,2,5,14,16,17,18,19,21,24,26,27,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(df.shape)
df.head()

(157453, 37)


,Unnamed: 0,all_awardings,approved_at_utc,author,author_cakeday,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,...,no_follow,parent_id,permalink,retrieved_on,score,send_replies,stickied,subreddit,subreddit_id,total_awards_received
0,0,[],NaN,HEROROYDEN,NaN,NaN,NaN,[],NaN,NaN,...,True,t1_evdwqpw,/r/AskReddit/comments/cjjtnn/people_of_reddit_...,2019-07-30 01:28:02,1.0,True,False,AskReddit,t5_2qh1i,0.0
1,1,[],NaN,marlashannon,NaN,NaN,NaN,[],NaN,NaN,...,True,t1_evdvpyz,/r/AskReddit/comments/cjjtnn/people_of_reddit_...,2019-07-30 01:17:41,1.0,True,False,AskReddit,t5_2qh1i,0.0
2,2,[],NaN,HEROROYDEN,NaN,NaN,NaN,[],NaN,NaN,...,True,t1_evdsoc2,/r/AskReddit/comments/cjjtnn/people_of_reddit_...,2019-07-30 01:05:12,1.0,True,False,AskReddit,t5_2qh1i,0.0
3,3,[],NaN,ahlannister,NaN,NaN,NaN,[],NaN,NaN,...,True,t3_cjjtnn,/r/AskReddit/comments/cjjtnn/people_of_reddit_...,2019-07-30 01:04:39,1.0,True,False,AskReddit,t5_2qh1i,0.0
4,4,[],NaN,Dobbeo,NaN,NaN,NaN,[],NaN,NaN,...,True,t3_cjjtnn,/r/AskReddit/comments/cjjtnn/people_of_reddit_...,2019-07-30 00:47:52,1.0,True,False,AskReddit,t5_2qh1i,0.0


In [4]:
df.columns

Index(['Unnamed: 0', 'all_awardings', 'approved_at_utc', 'author',
       'author_cakeday', 'author_flair_background_color',
       'author_flair_css_class', 'author_flair_richtext',
       'author_flair_template_id', 'author_flair_text',
       'author_flair_text_color', 'author_flair_type', 'author_fullname',
       'author_patreon_flair', 'banned_at_utc', 'body', 'can_mod_post',
       'collapsed', 'collapsed_reason', 'created_utc', 'distinguished',
       'edited', 'gildings', 'id', 'is_submitter', 'link_id', 'locked',
       'no_follow', 'parent_id', 'permalink', 'retrieved_on', 'score',
       'send_replies', 'stickied', 'subreddit', 'subreddit_id',
       'total_awards_received'],
      dtype='object')

In [5]:
df2 = df[['body', 'subreddit', 'retrieved_on']]

In [6]:
stopwords = spacy.lang.en.STOP_WORDS
stopwords |= {'\n\n\n', '\n\n', '\n', 'amp;#x200B', 'reddit', '/r', '/r/', 'r/', '%', '_',
              'like', 'that', 'this', 'r', 'askhistorians', 'if', 'the', 'thing', 'things',
              'question', 'questions', 'answer', 'answers', 'gt', '\&gt', 'wiki', 'subreddit',
              'askphilosophy', 'philosophy', 'op', 'comment', 'comments', 'account', 'repost',
              'notification', 'notifications', 'message', 'messages', 'lot', 'engineer', 
              'engineering', 'askscience', 'flair', 'moderator', 'mod', 'mods', 'askreddit',
              'askcomputerscience', 'askculinary', 'trueaskreddit', 'asksocialscience', 
              'askengineers', 'nan', 'search?q', 'thanks', 'ban', 'way'}

In [7]:
tokens = []
punctuations = string.punctuation
stopwords = spacy.lang.en.STOP_WORDS

def word_is_good(word):
    return ((word.text not in stopwords) and 
            (word.pos_ == 'NOUN') and
            (word.pos_ != 'PUNCT') and
            (word.pos_ != 'SYM') and
            (word.lemma_ != '-PRON-'))
    
for doc in nlp.pipe(df2['body'].astype('unicode').values, batch_size=50,
                        n_threads=4):
    if doc.is_parsed:
        tokens.append([n.text.lower() for n in doc if word_is_good(n)])

df2['comment_tokens'] = tokens

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
df2.head()

,body,subreddit,retrieved_on,comment_tokens
0,How is he a buffoon then you idiot.,AskReddit,2019-07-30 01:28:02,"[buffoon, idiot]"
1,I didn’t say racist. I said buffoon. Completel...,AskReddit,2019-07-30 01:17:41,[buffoon]
2,LMAO you are hilarious. How can I be this stup...,AskReddit,2019-07-30 01:05:12,"[remarks, rat, city, people, raciests]"
3,"Eat rice, beans and salad together at lunch",AskReddit,2019-07-30 01:04:39,"[rice, beans, salad, lunch]"
4,Consuming copious amount of cheese.,AskReddit,2019-07-30 00:47:52,[cheese]


In [9]:
subs = ['AskReddit', 'AskHistorians', 'AskCulinary', 'AskEngineers', 'askscience',
        'askphilosophy', 'TrueAskReddit', 'AskComputerScience', 'AskSocialScience']
for subreddit in subs:
    conditions = ((df2['comment_tokens'].str.len() > 0) & (df2['subreddit'] == subreddit))
    # A Dictionary Representation of all the words in our corpus
    id2word = corpora.Dictionary(df2[conditions]['comment_tokens'])

    # Let's remove extreme values from the dataset
    id2word.filter_extremes(no_below=10, no_above=0.85)

    corpus = [id2word.doc2bow(text) for text in df2[conditions]['comment_tokens']]
    
    lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=42,
                   num_topics = 5,
                   passes=5,
                   workers=4)
    
    words = [re.findall(r'"([^"]*)"', t[1]) for t in lda.print_topics()]
    topics = [' '.join(t[0:5]) for t in words]

    print('\n\n' + subreddit + '\n------------------')
    for topic in topics:
        print(topic)



AskReddit
------------------
day life guy night time
shrek man friend years shit
people water food music fuck
person money year job time
time people friends life game


AskHistorians
------------------
history time women people period
war time thanks people sources
people time century word fact
people time war century point
time depth on&amp;sort interest rules


AskCulinary
------------------
bread recipe cooking baking recipes
salt cream rice iron pepper
time chicken chocolate recipe oven
sauce meat dough flavor water
water heat pan thanks egg


AskEngineers
------------------
job work company time experience
system power heat air pressure
job software position day cars
school time job people degree
people years work job engineers


askscience
------------------
language post save information games
time cells years people life
post forum = concerns action
body brain species effect sugar
water energy air temperature light


askphilosophy
------------------
people p example problem c